## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-22-02-19-12 +0000,bbc,Hurricane Erin to drench parts of Virginia whi...,https://www.bbc.com/news/articles/cwye3elwxd9o...
1,2025-08-22-02-18-01 +0000,nyt,Judge Orders ‘Alligator Alcatraz’ Detention Ce...,https://www.nytimes.com/2025/08/21/us/alligato...
2,2025-08-22-02-17-10 +0000,nypost,"Japan ‘bear attack’ victim, 93, was actually m...",https://nypost.com/2025/08/21/world-news/japan...
3,2025-08-22-02-08-00 +0000,wsj,Erik Menendez’s Parole Bid Is Rejected,https://www.wsj.com/us-news/law/erik-menendezs...
4,2025-08-22-02-07-00 +0000,wsj,Japan’s Inflation Stays Well Above BOJ’s 2% Ta...,https://www.wsj.com/economy/japans-inflation-s...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2297/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
71,trump,53
358,new,23
209,ukraine,15
18,was,13
162,court,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
69,2025-08-21-22-35-03 +0000,startribune,Appeals court throws out massive civil fraud p...,https://www.startribune.com/appeals-court-thro...,113
129,2025-08-21-19-17-34 +0000,nyt,NY Appeals Court Throws Out Trump’s Half-Billi...,https://www.nytimes.com/2025/08/21/nyregion/tr...,112
111,2025-08-21-20-32-00 +0000,wsj,N.Y. Appeals Court Throws Out $500 Million Civ...,https://www.wsj.com/us-news/law/ny-appeals-cou...,110
72,2025-08-21-22-28-59 +0000,bbc,Appeals court throws out Trump's $500m civil f...,https://www.bbc.com/news/articles/c5y09q1zgg8o...,103
119,2025-08-21-20-11-18 +0000,nypost,Trump pick Alina Habba is unlawfully serving a...,https://nypost.com/2025/08/21/us-news/trump-pi...,101


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
69,113,2025-08-21-22-35-03 +0000,startribune,Appeals court throws out massive civil fraud p...,https://www.startribune.com/appeals-court-thro...
52,69,2025-08-21-23-43-37 +0000,nypost,California Gov. Gavin Newsom signs redistricti...,https://nypost.com/2025/08/21/us-news/californ...
177,60,2025-08-21-17-07-24 +0000,nypost,North Korea’s top officers abruptly pulled out...,https://nypost.com/2025/08/21/world-news/north...
261,53,2025-08-21-12-30-03 +0000,nypost,US keeping lofty EU car tariffs until levies o...,https://nypost.com/2025/08/21/us-news/us-keepi...
7,50,2025-08-22-02-00-24 +0000,wapo,"Erik Menendez, who was convicted of killing hi...",https://www.washingtonpost.com/nation/2025/08/...
59,49,2025-08-21-23-02-09 +0000,nypost,Israeli stats show civilian death rate in Gaza...,https://nypost.com/2025/08/21/world-news/israe...
181,43,2025-08-21-17-01-27 +0000,nypost,"Trump says he’ll patrol DC with cops, military...",https://nypost.com/2025/08/21/us-news/trump-sa...
237,42,2025-08-21-14-13-25 +0000,cbc,"Judge Frank Caprio, known online as the 'nices...",https://www.cbc.ca/news/world/judge-frank-capr...
90,36,2025-08-21-21-46-50 +0000,nypost,Murdered NJ vet Lauren Semanchik left voicemai...,https://nypost.com/2025/08/21/us-news/lauren-s...
86,36,2025-08-21-22-03-00 +0000,wsj,Two Fed officials offered competing assessment...,https://www.wsj.com/economy/central-banking/fe...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
